In [1]:
import pandas as pd
import numpy as np

soil = pd.read_csv("data/raw/soil_samples.csv")
traits = pd.read_csv("data/raw/tree_species_traits.csv")

soil.head(), traits.head()


(  plot_id  nitrogen_pct  phosphorus_ppm  potassium_ppm  soil_carbon_pct    ph  \
 0    P001         0.212              22            132             3.67  6.10   
 1    P002         0.385              16            147             5.53  6.47   
 2    P003         0.320               6            250             3.30  5.25   
 3    P004         0.280              14            161             2.49  5.01   
 4    P005         0.147               8            136             3.43  6.14   
 
   erosion_risk  
 0          low  
 1       medium  
 2       medium  
 3          low  
 4       medium  ,
    species_id  wood_density_g_cm3  max_height_m  leaf_area_cm2  seed_mass_g  \
 0  Species_01               0.629          33.1          168.8         8.35   
 1  Species_02               0.629          24.2          192.6        17.28   
 2  Species_03               0.421          25.7           81.7        18.47   
 3  Species_04               0.711          31.3           60.9         9.37 

In [2]:
np.random.seed(123)

plot_ids = soil["plot_id"].tolist()
species_ids = traits["species_id"].tolist()

rows = []
for plot in plot_ids:
    # choose 5–10 species per plot
    n_sp = np.random.randint(5, 11)
    chosen = np.random.choice(species_ids, size=n_sp, replace=False)
    for sp in chosen:
        abundance = np.random.randint(1, 50)
        rows.append({"plot_id": plot, "species_id": sp, "abundance": abundance})

community = pd.DataFrame(rows)
community.head()


,plot_id,species_id,abundance
0,P001,Species_07,22
1,P001,Species_11,31
2,P001,Species_13,28
3,P001,Species_39,35
4,P001,Species_06,34


In [3]:
from math import log

def shannon_index(group):
    total = group["abundance"].sum()
    if total == 0:
        return 0.0
    proportions = group["abundance"] / total
    return -sum(p * log(p) for p in proportions)

diversity = (
    community
    .groupby("plot_id")
    .apply(lambda g: pd.Series({
        "richness": g["species_id"].nunique(),
        "shannon": shannon_index(g)
    }))
    .reset_index()
)

diversity.head()


/tmp/ipykernel_1316/234087051.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,plot_id,richness,shannon
0,P001,10.0,2.141606
1,P002,5.0,1.389333
2,P003,6.0,1.420326
3,P004,9.0,2.006177
4,P005,6.0,1.622051


In [6]:
def health_score(row):
    carbon_score = (row["soil_carbon_pct"] - 2.0) / (6.0 - 2.0)
    ph_score = 1.0 - abs(row["ph"] - 6.0) / 2.0
    return max(0.0, min(1.0, 0.5 * carbon_score + 0.5 * ph_score))

soil["health_score"] = soil.apply(health_score, axis=1)
soil[["plot_id", "soil_carbon_pct", "ph", "health_score"]].head()



,plot_id,soil_carbon_pct,ph,health_score
0,P001,3.67,6.10,0.68375
1,P002,5.53,6.47,0.82375
2,P003,3.30,5.25,0.47500
3,P004,2.49,5.01,0.31375
4,P005,3.43,6.14,0.64375


In [7]:
diversity_health = diversity.merge(soil[["plot_id", "health_score"]], on="plot_id")
diversity_health.head()


,plot_id,richness,shannon,health_score
0,P001,10.0,2.141606,0.68375
1,P002,5.0,1.389333,0.82375
2,P003,6.0,1.420326,0.47500
3,P004,9.0,2.006177,0.31375
4,P005,6.0,1.622051,0.64375


In [8]:
diversity_health[["health_score", "richness", "shannon"]].corr()


,health_score,richness,shannon
health_score,1.000000,-0.105427,-0.116999
richness,-0.105427,1.000000,0.935018
shannon,-0.116999,0.935018,1.000000
